In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [19]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [25]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
logits
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1).item()
print(f"Predicted class: {y_pred}")


Predicted class: 6


In [27]:
input_image = torch.rand(3, 28, 28)
input_image.size()

torch.Size([3, 28, 28])

In [29]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.size()

torch.Size([3, 784])

In [34]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
hidden1.size()

torch.Size([3, 20])

In [36]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.0000, 0.1675, 0.0000, 0.0000, 0.4016, 0.1840, 0.0000, 0.9477, 0.0000,
         0.0000, 0.0000, 0.0000, 0.2797, 0.2449, 0.1980, 0.0000, 0.2142, 0.7996,
         0.5218, 0.0267],
        [0.0000, 0.2628, 0.0000, 0.0000, 0.4006, 0.1677, 0.0000, 0.3588, 0.1254,
         0.0000, 0.0000, 0.0000, 0.0273, 0.0000, 0.0000, 0.0386, 0.1358, 0.3342,
         0.0000, 0.0000],
        [0.0000, 0.1041, 0.0334, 0.0000, 0.3790, 0.0000, 0.0000, 0.5677, 0.0346,
         0.0000, 0.0000, 0.0000, 0.0891, 0.0691, 0.1799, 0.0000, 0.2063, 0.9770,
         0.1704, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0000, 0.1675, 0.0000, 0.0000, 0.4016, 0.1840, 0.0000, 0.9477, 0.0000,
         0.0000, 0.0000, 0.0000, 0.2797, 0.2449, 0.1980, 0.0000, 0.2142, 0.7996,
         0.5218, 0.0267],
        [0.0000, 0.2628, 0.0000, 0.0000, 0.4006, 0.1677, 0.0000, 0.3588, 0.1254,
         0.0000, 0.0000, 0.0000, 0.0273, 0.0000, 0.0000, 0.0386, 0.1358, 0.3342,
         0.0000, 0.0000],
       

In [44]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
logits

tensor([[ 0.1971,  0.1029, -0.0535, -0.1516,  0.1379, -0.0631,  0.0278,  0.0770,
         -0.2513, -0.2982],
        [ 0.1849,  0.1487, -0.0059, -0.1659,  0.1041, -0.0357,  0.0543,  0.1103,
         -0.3095, -0.2971],
        [ 0.2163,  0.1967,  0.0780, -0.0248,  0.2203,  0.0833, -0.0343,  0.1808,
         -0.2367, -0.4303]], grad_fn=<AddmmBackward0>)

In [46]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_class = pred_probab.argmax(dim=1)
pred_probab, pred_class

(tensor([[0.1237, 0.1125, 0.0963, 0.0873, 0.1166, 0.0953, 0.1044, 0.1097, 0.0790,
          0.0754],
         [0.1212, 0.1169, 0.1001, 0.0853, 0.1118, 0.0972, 0.1063, 0.1125, 0.0739,
          0.0748],
         [0.1188, 0.1165, 0.1034, 0.0933, 0.1192, 0.1040, 0.0924, 0.1146, 0.0755,
          0.0622]], grad_fn=<SoftmaxBackward0>),
 tensor([0, 0, 4]))

In [50]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0318,  0.0284,  0.0216,  ..., -0.0133, -0.0265,  0.0299],
        [-0.0230, -0.0154,  0.0151,  ...,  0.0091,  0.0274,  0.0304]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0288, -0.0347], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0322,  0.0125,  0.0098,  ..., -0.0255,  0.0406, -0.0144],
        [-0.0412,  0.0068,  0.0148,  ...,  0.0290, -0.0090, -0.0282]],
       device='cuda:0', grad_fn=<SliceBa